In [1]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import time
import random

In [ ]:
"""
    Legal Issues:
        Don't worry about it.
        But keep note, I should respect robots.txt
        But if I do collect the data, DO NOT give it to others.
        Just don't sell or advertise that the data is mine.
        Don't mess with financial or gov data.
        Don't infringe on actual copyright stuff.
        Don't enter anything that requires permission, passwords.
        Don't scrape emails, usernames.
        Don't spam forms.
"""

"""
    Dealing With HTTP Errors:
        Don't move scrapers too quickly.
        Change headers.
        Don't anything a human wouldn't.
"""

"""
    Scraping Remotely:
        Use TOR browser (bounces IP address)
        DuckDuckGo does not store cookies
        Scrape on Google Cloud. Has access to changing IP addresses.
"""

In [2]:
session = requests.Session()
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
           'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}

In [ ]:
'''
    Go to starting page.
    Gather all the links in that one page.
    Go to next page.
    Repeat.
    Stop when the page does not exist. 
        Problem: the page exists even when it shouldn't
        Example: searching &start=1000 seems to produce the first page
        Solution: need to create a numpy array of links,
            append the new links if the new links are not found
            stop if the links are found
        Current Solution (going to change this): 
            Specify a limit
'''

In [ ]:
list_of_Chicago_Zip = [60007, 60018, 60068, 60106, 60131, 60176, 60601, 60602, 60603, 60604, 60605, 60606, 60607, 60608, 60609, 60610, 60611, 60612, 60613, 60614, 60615, 60616, 60617, 60618, 60619, 60620, 60621, 60622, 60623, 60624, 60625, 60626, 60628, 60629, 60630, 60631, 60632, 60633, 60634, 60636, 60637, 60638, 60639, 60640, 60641, 60642, 60643, 60644, 60645, 60646, 60647, 60649, 60651, 60652, 60653, 60654, 60655, 60656, 60657, 60659, 60660, 60661, 60706, 60707, 60714, 60804, 60827]

In [3]:
# Don't use this one, use the one below
def gather_Yelp_data(url, cap_num):
    """
        Given a starting url and a page limit,
        gather all the links to businesses.
        Go to the next page.
        Gather all links.
        Repeat a specified number of times.
        Return a numpy array of all links gathered.
    """
    item_number = 0
    websites = np.array([])
    while item_number <= cap_num:
        try:
            #html = urlopen(url + "&start=" + str(item_number*10))
            #bs = BeautifulSoup(html.read(), 'lxml')
            req = session.get(url + "&start=" + str(item_number*10), headers=headers)
            bs = BeautifulSoup(req.text, 'lxml')
            links = bs.findAll('a', {'class': "biz-name js-analytics-click"})
            for link in links:
                #print (link)
                link_string = link.attrs['href']
                websites = np.append(websites, np.array([link_string]))
        except:
            pass
        time.sleep (random.random())
        item_number += 1
    return websites
starturl = """https://www.yelp.com/search?find_desc=cafe&find_loc=60637"""

start_time = time.time()
sites = gather_Yelp_data(starturl, 0)
print("--- %s seconds ---" % (time.time() - start_time))
sites

--- 0.9090261459350586 seconds ---


array([], dtype=float64)

In [3]:
"""
    Huge Problem of Inconsistency:
    Sometimes things are returned, others nothing is returned. 
    Why? I don't know. Running the same thing returns different results.
    So how do I know when I'm not getting any result? Label Them
"""
def gather_Yelp_data(url, cap_num, page_num):
    instance = 0
    item_number = page_num
    websites = []
    num_label = []
    while instance <= cap_num:
        try:
            req = session.get(url + "&start=" + str(item_number*10), headers=headers)
            bs = BeautifulSoup(req.text, 'lxml')
            links = bs.findAll('a', {'class': "biz-name js-analytics-click"})
            for link in links:
                link_string = link.attrs['href']
                websites.append (link_string)
                num_label.append (item_number)
        except:
            pass
        time.sleep (random.random())
        item_number += 1
        instance += 1
    return (pd.DataFrame ({'Link':websites, 'Num': num_label}), item_number)
starturl = """https://www.yelp.com/search?find_desc=cafe&find_loc=chicago"""

yelp_page_num = 0
start_time = time.time()
sites, yelp_page_num = gather_Yelp_data(starturl, 2, yelp_page_num)
print("--- %s seconds ---" % (time.time() - start_time))
sites

--- 8.08768105506897 seconds ---


,Link,Num
0,/adredir?ad_business_id=hydni_IzDW29tj9xdnyQnw...,0
1,/adredir?ad_business_id=oWKyW2sx03CLFH1bJOYGGg...,0
2,/biz/the-wormhole-coffee-chicago?osq=cafe,0
3,/biz/limitless-coffee-fulton-market-chicago?os...,0
4,/biz/the-avondale-coffee-club-chicago?osq=cafe,0
5,/biz/hero-coffee-bar-chicago-14?osq=cafe,0
6,/biz/tea-pot-brew-bakery-chicago?osq=cafe,0
7,/biz/purple-llama-chicago-3?osq=cafe,0
8,/biz/electric-mud-coffee-bar-chicago-2?osq=cafe,0
9,/biz/intelligentsia-coffee-chicago-8?osq=cafe,0


In [44]:
sites.to_csv('websites.csv', mode='w')

In [68]:
yelp_page_num = 100

In [69]:
start_time = time.time()
sites, yelp_page_num = gather_Yelp_data(starturl, 20, yelp_page_num)
print("--- %s seconds ---" % (time.time() - start_time))
sites
sites.to_csv('websites.csv', mode='a')

--- 18.866631031036377 seconds ---


In [70]:
yelp_page_num

121

In [ ]:
'''
    Going to have to search by zip code.
    There are 42000 zip codes in US.
    How do we find the number of cafes in Yelp?
    Take a random sample (say 30) of zip codes.
    Search Yelp through the Zip Code.
    Find average number of businesses. 
    Multiply that by 42000
'''

'''
import random
for x in range(30):
  print (random.randint(10000,99999)) 
'''

num_cafes = [
    21, 10, 69, 33, 21, 55, 18, 39, 16, 47, 16, 24, 16, 37, 28, 26, 8, 37, 22, 5, 81
]
average_num_Cafe = sum(num_cafes) / len (num_cafes) # 29.952380952380953
num_cafe_in_US = average_num_Cafe * 43000 # 1287952

'''
    About 1.945 seconds per visited webpage (10 business for one webpage)
    Then about 69.6 hours for visiting all webpages and collecting websites.
    
    About 7.5 seconds per taking all review pages of a single business.
    About 3.3 seconds to take all additional info.
    10.8 seconds total.
    So then 161 days.
    
'''
print (num_cafe_in_US)

--- 3.834855079650879 seconds ---


{'Accepts Apple Pay': 'No',
 'Accepts Credit Cards': 'Yes',
 'Accepts Google Pay': 'No',
 'Address': '501 E 61st StChicago, IL 60637',
 'Bike Parking': 'Yes',
 'Caters': 'No',
 'Gender Neutral Restrooms': 'Yes',
 'Good for Working': 'Yes',
 'Health Score': '95 out of 100',
 'Outdoor Seating': 'Yes',
 'Parking': 'Street, Private Lot',
 'Price Range': 'Inexpensive',
 'Take-out': 'Yes',
 'Wheelchair Accessible': 'Yes',
 'Wi-Fi': 'Free'}

--- 9.250655174255371 seconds ---


,Comment,Date,Star Ratings
0,Excited to find this chill space! Great friend...,\n 7/14/2018\n,5.0 star rating
1,This place is AWESOME!I was so hungry I gobble...,\n 9/8/2017\n,5.0 star rating
2,I love this place and what it's trying to do f...,\n 4/13/2018\n,5.0 star rating
3,This winter season has been very mild in compa...,\n 2/23/2016\n,4.0 star rating
4,"Very good coffee. Affordable and local, cant b...",\n 1/8/2018\n,5.0 star rating
5,I really enjoy bringing my work and sipping an...,\n 6/22/2017\n,5.0 star rating
6,"An oasis in the dessert! This is a nice, clean...",\n 5/19/2018\n,5.0 star rating
7,Pros: Very friendly ambiance especially if you...,\n 3/10/2016\n,5.0 star rating
8,This place is a great local coffee shop locate...,\n 8/9/2017\n,4.0 star rating
9,My coffee (caramel macchiato)tasted exactly th...,\n 8/18/2017\n,1.0 star rating


In [ ]:
greenline_coffee_reviews.to_csv('Greenline Coffee Reviews.csv')